In [1]:
# python standard library
import os
from datetime import datetime

# pypi
import holoviews
import numpy
import pandas

# local
from common import (
    download_data,
    Paths,
    Urls,
    )

In [2]:
portland = download_data(Paths.portland, Urls.portland)

In [3]:
column_renames = {"Value": "unemployment_rate",
                  "Label": "date"}
portland.rename(columns=column_renames,
                inplace=True)

In [4]:
month_map = dict(M01="Jan", M02="Feb", M03="Mar", M04="Apr", M05="May",
                 M06="Jun", M07="Jul", M08="Aug", M09="Sep", M10="Oct",
                 M11="Nov", M12="Dec")
portland["month"] = portland.Period.apply(lambda x: month_map[x])

In [5]:
month_integers = dict(zip("Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec".split(), range(1, 13)))

In [7]:
portland["datetime"] = portland.apply(lambda row: datetime(row.Year, month_integers[row.month], 1), axis=1)

In [8]:
national = download_data(Paths.national, Urls.national)

In [9]:
national.rename(columns=column_renames, inplace=True)
national["month"] = national.Period.apply(lambda x: month_map[x])

In [10]:
national.drop([122], inplace=True)

In [11]:
national["datetime"] = national.apply(lambda row: datetime(row.Year, month_integers[row.month], 1),
                                      axis=1)

In [12]:
s_and_p_index = download_data(Paths.s_and_p, Urls.s_and_p, na_values=".")

In [13]:
pre = pandas.DataFrame({"DATE": ["2007-01-01", "2007-02-01", "2007-03-01"], "VALUE": [numpy.nan, numpy.nan, numpy.nan]})
s_and_p_index = pre.append(s_and_p_index)
s_and_p_index["date"] = portland.date.values
s_and_p_index = s_and_p_index.reset_index(drop=True)

In [14]:
s_and_p_index["year"] = s_and_p_index.date.apply(lambda row: int(row.split()[0]))
s_and_p_index["month"] = s_and_p_index.date.apply(lambda row: row.split()[1])

In [15]:
s_and_p_index["datetime"] = s_and_p_index.apply(lambda row: datetime(row.year,
                                                                     month_integers[row.month], 1),
                                                    axis=1)

In [16]:
house_price_index = download_data(Paths.house_price, Urls.house_price)

In [17]:
house_price_index["price"] = house_price_index.HPIPONM226S
house_price_index["date"] = portland.date[1:].values
pre = pandas.DataFrame({"DATE": ["2007-01-01"], "HPIPONM226S": [numpy.nan], "price": [numpy.nan], "date": ["2007 Jan"]})
house_price_index = pre.append(house_price_index)
house_price_index = house_price_index.reset_index(drop=True)

In [18]:
house_price_index["year"] = house_price_index.date.apply(
    lambda row: int(row.split()[0]))
house_price_index["month"] = house_price_index.date.apply(
    lambda row: row.split()[1])
house_price_index["datetime"] = house_price_index.apply(
    lambda row: datetime(row.year,
                         month_integers[row.month], 1),
    axis=1)

In [19]:
highest_unemployment = portland.unemployment_rate.max()
unemployment_peaks = numpy.where(portland.unemployment_rate==highest_unemployment)[0]

In [20]:
lowest_unemployment = portland.unemployment_rate.min()

In [21]:
recession_start = numpy.where(portland.date=="2007 Dec")[0][0]
recession_end = numpy.where(portland.date=="2009 Jun")[0][0]
portland_recession_start = portland.unemployment_rate.iloc[recession_start]

In [22]:
peak = national.unemployment_rate.max()
national_peak = numpy.where(national.unemployment_rate==peak)
national_recession_start = national.unemployment_rate.iloc[recession_start]
post_recession = national[national.Year > 2009]
index = numpy.where(post_recession.unemployment_rate==national_recession_start)[0][0]

In [23]:
s_and_p_nadir = s_and_p_index.VALUE.min()
s_and_p_nadir = numpy.where(s_and_p_index.VALUE==s_and_p_nadir)[0]

In [24]:
housing_nadir = house_price_index.price.min()
housing_nadir = numpy.where(house_price_index.price==housing_nadir)[0]

In [25]:
NATIONAL_COLOR = "slategrey"
NATIONAL_LABEL = "National"
PORTLAND_COLOR = "cornflowerblue"
PORTLAND_LABEL = "Portland-Hillsboro-Vancouver"
S_AND_P_COLOR = "#90151B"
S_AND_P_LABEL = "S & P 500 Index"
HOUSING_COLOR = "#D89159"
HOUSING_LABEL = "House Price Index"

In [26]:
from bokeh.models import (
    BoxAnnotation,
    CustomJS,
    Span,
    Toggle,
    )

In [28]:
from bokeh.io import (
    output_file,
    output_notebook,
    show,
    )

from bokeh.plotting import (
    figure,
    ColumnDataSource,
    )

from bokeh.models import (
    CrosshairTool,
    HoverTool,
    PanTool,
    ResetTool,
    ResizeTool,
    SaveTool,
    UndoTool,
    WheelZoomTool,
    )

In [31]:
from bokeh.layouts import column

In [32]:
FIGURE_WIDTH = 600
FIGURE_HEIGHT = 300

In [29]:
s_and_p_source = ColumnDataSource(
    data=dict(
        month_data=s_and_p_index.datetime,
        value=s_and_p_index.VALUE,
        month_label=s_and_p_index.date,
        )
)

In [30]:
housing_source = ColumnDataSource(
    data=dict(
        month_data=house_price_index.datetime,
        price=house_price_index.price,
        month_label=s_and_p_index.date,
        )
)

In [33]:
def make_tools():
    """makes the tools for the figures
    
    Returns:
     list: tool objects
    """
    hover = HoverTool(tooltips=[
    ("month", "@month_label"),
    ("unemployment", "@unemployment"),
    ])
    
    tools = [
        hover,
        CrosshairTool(),
        PanTool(),
        ResetTool(),
        ResizeTool(),
        SaveTool(),
        UndoTool(),
        WheelZoomTool(),
    ]
    return tools

In [34]:
tools = make_tools()
unemployment_figure = figure(
    plot_width=FIGURE_WIDTH,
    plot_height=FIGURE_HEIGHT,
    x_axis_type="datetime",
    tools=tools,
    title="Portland Unemployment (2007-2017)"
)

In [36]:
portland_source = ColumnDataSource(
    data=dict(
        month_data=portland.datetime,
        unemployment=portland.unemployment_rate,
        month_label=portland.date,
        )
)

In [38]:
national_source = ColumnDataSource(
    data=dict(
        month_data=national.datetime,
        unemployment=national.unemployment_rate,
        month_label=national.date,
        )
)

In [39]:
unemployment_figure.line(
    "month_data", "unemployment",
    source=portland_source,
    line_color=PORTLAND_COLOR,
    legend=PORTLAND_LABEL,
          )

line = unemployment_figure.line(
    "month_data", "unemployment",
    source=national_source,
    line_color=NATIONAL_COLOR,
    legend=NATIONAL_LABEL,
)

In [40]:
TIME_SCALE = 10**3

In [41]:
def scale_timestamp(index):
    """gets the scaled timestamp for element location

    Args:
     index: index in the portland.datetime series
    Returns:
     epoch timestamp used to locate place in plot
    """
    return portland.datetime[index].timestamp() * TIME_SCALE

In [42]:
def make_recession():
    """Makes the box for the recession

    Returns:
     BoxAnnotation to color the recession
    """
    return BoxAnnotation(
        left=scale_timestamp(recession_start),
        right=scale_timestamp(recession_end),
        fill_color="blue",
        fill_alpha=0.1)

In [43]:
def make_vertical(location, color="darkorange"):
    """makes a vertical line
    
    Args:
     location: place on the x-axis for the line
     color (str): line-color for the line
    Returns:
     Span at index
    """
    return Span(
        location=location,
        line_color=color,
        dimension="height",
    )

In [44]:
def make_verticals(fig):
    """makes the verticals and adds them to the figures"""
    fig.add_layout(make_vertical(
        location=scale_timestamp(unemployment_peaks[0]),
        color="darkorange",
    ))
    fig.add_layout(make_vertical(
        location=scale_timestamp(s_and_p_nadir[0]),
        color="crimson"))
    fig.add_layout(make_vertical(
        location=scale_timestamp(housing_nadir[0]),
        color="limegreen"))
    fig.add_layout(make_vertical(
        location=scale_timestamp(national_peak[0][0]),
        color="grey"))
    return

In [45]:
unemployment_figure.add_layout(make_recession())

In [46]:
make_verticals(unemployment_figure)

In [47]:
unemployment_figure.yaxis.axis_label = "% Unemployment"
unemployment_figure.xaxis.axis_label = "Month"
unemployment_figure.xgrid.visible = False
unemployment_figure.ygrid.visible = False

In [48]:
hover = HoverTool(tooltips=[
    ("Month", "@month_label"),
    ("Value", "@value"),
])
tools = [
    hover,
    CrosshairTool(),
    PanTool(),
    ResetTool(),
    ResizeTool(),
    SaveTool(),
    UndoTool(),
    WheelZoomTool(),
]
s_and_p_figure = figure(
    plot_width=FIGURE_WIDTH,
    plot_height=FIGURE_HEIGHT,
    x_range=unemployment_figure.x_range,
    x_axis_type="datetime",
    tools=tools,
    title="S & P 500 Index",
)
line = s_and_p_figure.line("month_data", "value",
                    source=s_and_p_source,
                    line_color=S_AND_P_COLOR)

In [49]:
s_and_p_figure.add_layout(make_recession())

In [50]:
make_verticals(s_and_p_figure)

In [51]:
s_and_p_figure.yaxis.axis_label = "S & P 500 Valuation"
s_and_p_figure.xaxis.axis_label = "Month"
s_and_p_figure.xgrid.visible = False
s_and_p_figure.ygrid.visible = False

In [52]:
s_and_p_figure.legend.location = "bottom_right"

In [53]:
hover = HoverTool(tooltips=[
    ("Month", "@month_label"),
    ("Price", "@price"),
])
tools = [
    hover,
    CrosshairTool(),
    PanTool(),
    ResetTool(),
    ResizeTool(),
    SaveTool(),
    UndoTool(),
    WheelZoomTool(),
]
housing_figure = figure(
    plot_width=FIGURE_WIDTH,
    plot_height=FIGURE_HEIGHT,
    x_range=unemployment_figure.x_range,
    x_axis_type="datetime",
    tools=tools,
    title="House Price Index",
)
line = housing_figure.line("month_data", "price",
                           source=housing_source,
                           line_color=HOUSING_COLOR)

In [54]:
housing_figure.add_layout(make_recession())
make_verticals(housing_figure)

In [55]:
housing_figure.yaxis.axis_label = "Sale Price ($1,000)"
housing_figure.xaxis.axis_label = "Month"
housing_figure.xgrid.visible = False
housing_figure.ygrid.visible = False

In [56]:
housing_figure.legend.location = "bottom_right"

In [57]:
combined = column(unemployment_figure, s_and_p_figure, housing_figure)

In [58]:
from bokeh.resources import CDN
from bokeh.embed import autoload_static

In [59]:
OUTPUT_JAVASCRIPT = "portland_unemployment.js"

In [60]:
js, tag = autoload_static(combined, CDN, OUTPUT_JAVASCRIPT)

In [61]:
with open(OUTPUT_JAVASCRIPT, "w") as writer:
    writer.write(js)

In [62]:
with open("portland_tag.html", 'w') as writer:
    writer.write(tag)